In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn import svm
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [2]:
#dataset sa benignim url-ovima
ds_benign= pd.read_csv('https://raw.githubusercontent.com/matf-ml/ml19-classifying-spam-urls/master/Benign_list_big_final.csv', header = None)
ds_benign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35378 entries, 0 to 35377
Data columns (total 1 columns):
0    35378 non-null object
dtypes: object(1)
memory usage: 276.5+ KB


In [3]:
ds_benign.columns = ['url']
ds_benign['label'] = 0
ds_benign.head()

,url,label
0,http://1337x.to/torrent/1048648/American-Snipe...,0
1,http://1337x.to/torrent/1110018/Blackhat-2015-...,0
2,http://1337x.to/torrent/1122940/Blackhat-2015-...,0
3,http://1337x.to/torrent/1124395/Fast-and-Furio...,0
4,http://1337x.to/torrent/1145504/Avengers-Age-o...,0


In [4]:
#dataset sa spam url-ovima
ds_spam = pd.read_csv('https://raw.githubusercontent.com/matf-ml/ml19-classifying-spam-urls/master/spam_dataset.csv', header = None)
ds_spam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 1 columns):
0    12000 non-null object
dtypes: object(1)
memory usage: 93.8+ KB


In [5]:
ds_spam.columns = ['url']
ds_spam['label'] = 1
ds_spam.head()

,url,label
0,http://astore.amazon.co.uk/allezvinsfrenchr/de...,1
1,http://archive.salisburyjournal.co.uk/2007/3/6...,1
2,http://appbasic.jettons.co.uk/links/index.html,1
3,http://archive.yorkpress.co.uk/2003/11/6/25684...,1
4,http://acard4u.co.uk/product_reviews.php?cPath...,1


In [6]:
#dataset sa malware url-ovima
ds_malware = pd.read_csv('https://raw.githubusercontent.com/matf-ml/ml19-classifying-spam-urls/master/Malware_dataset.csv', header = None)
ds_malware.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11566 entries, 0 to 11565
Data columns (total 1 columns):
0    11566 non-null object
dtypes: object(1)
memory usage: 90.4+ KB


In [7]:
ds_malware.columns = ['url']
ds_malware['label'] = 1
ds_malware.head()

,url,label
0,http://gzzax.livechatvalue.com/chat/chatClient...,1
1,http://gzzax.livechatvalue.com/chat/chatClient...,1
2,http://gzzax.livechatvalue.com/chat/chatClient...,1
3,http://gzzax.livechatvalue.com/chat/chatClient...,1
4,http://mtsx.com.cn/UploadFiles/2011-08/admin/%...,1


In [8]:
#dataset sa phishing url-ovima
ds_phishing = pd.read_csv('https://raw.githubusercontent.com/matf-ml/ml19-classifying-spam-urls/master/phishing_dataset.csv', header = None)
ds_phishing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9965 entries, 0 to 9964
Data columns (total 1 columns):
0    9965 non-null object
dtypes: object(1)
memory usage: 77.9+ KB


In [9]:
ds_phishing.columns = ['url']
ds_phishing['label'] = 1
ds_phishing.head()

,url,label
0,http://v2.email-marketing.adminsimple.com/trac...,1
1,http://bid.openx.net/json?amp;amp;amp;amp;cid;...,1
2,http://webmail2.centurytel.net/hwebmail/servic...,1
3,http://www.google.com.ng/imgres?imgurl=http://...,1
4,http://webmail2.centurytel.net/hwebmail/servic...,1


In [10]:
#dataset sa defacement url-ovima
ds_defacement = pd.read_csv('https://raw.githubusercontent.com/matf-ml/ml19-classifying-spam-urls/master/DefacementSitesURLFiltered.csv', header = None)
ds_defacement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96457 entries, 0 to 96456
Data columns (total 1 columns):
0    96457 non-null object
dtypes: object(1)
memory usage: 753.6+ KB


In [11]:
ds_defacement.columns = ['url']
ds_defacement['label'] = 1
#shuffle dataseta da bi dobili raznovrsnije podatke, jer cemo posle uzeti samo 10k od oko 90k redova iz njega
ds_defacement = ds_defacement.sample(frac=1).reset_index(drop=True)
ds_defacement.head()

,url,label
0,http://www.muzikantaimaks.lt/index.php/compone...,1
1,http://www.maryahprincess.it/index.php?option=...,1
2,http://www.winnerhydraulicsusa.com/webstore/st...,1
3,http://www.thewrightway.me.uk/events.html,1
4,http://www.hauskrankenpflege-duckstein.de/inde...,1


In [12]:
ds_comb = ds_benign.append(ds_spam)
ds_comb = ds_comb.append(ds_malware)
ds_comb = ds_comb.append(ds_phishing)
#uzimamo 10k redova iz defacement dataset-a jer je preveliki, a hocemo da sacuvamo odnos losih i dobrih url-ova priblizno 50-50
ds_defacement = ds_defacement[:10000]
ds_comb = ds_comb.append(ds_defacement)
ds_comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78909 entries, 0 to 9999
Data columns (total 2 columns):
url      78909 non-null object
label    78909 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.8+ MB


In [13]:
#shuffle dataseta da bi videli raznovrsnije podatke
ds_comb = ds_comb.sample(frac=1).reset_index(drop=True)
ds_comb.head()

,url,label
0,http://aolshop.memberoffers.co.uk/csp/smp/aol/...,1
1,http://torcache.net/torrent/304E789D6E0E877BFD...,0
2,http://amber.ch.ic.ac.uk/archive/all/21291.html,1
3,http://www.gxyzx.com/js/?http://us.battle.net/...,1
4,http://online.citicard.com.rpzmvfcek2.agencja-...,1


In [0]:
import tldextract

In [15]:
def get_deep_url_from_url(url):
  path = urlparse(url)
  if path.query:
    return str(path.path) + str('?') + str(path.query)
  else:
    return str(path.path)

ds_comb['deep url'] = ds_comb['url'].apply(lambda x : get_deep_url_from_url(x))
ds_comb['len deep url'] = ds_comb['deep url'].apply(lambda x : len(x))
ds_comb.head()

,url,label,deep url,len deep url
0,http://aolshop.memberoffers.co.uk/csp/smp/aol/...,1,/csp/smp/aol/itementry3.csp?FS=E179&cat=AK&hea...,150
1,http://torcache.net/torrent/304E789D6E0E877BFD...,0,/torrent/304E789D6E0E877BFD6070B73082DE6B1FF46...,138
2,http://amber.ch.ic.ac.uk/archive/all/21291.html,1,/archive/all/21291.html,23
3,http://www.gxyzx.com/js/?http://us.battle.net/...,1,/js/?http://us.battle.net/login/en/?ref=http%3...,106
4,http://online.citicard.com.rpzmvfcek2.agencja-...,1,/doctorsincanton.com/wp-css/online.php,38


In [0]:
#radimo split deep url-a na reci da bi koristili to kao feature za MultinomialNB
import re
def split_deep_url(url):
    word_list = re.compile(r'[\:/?=\-&$~<>;%_\.\\]+',re.UNICODE).split(url)
    #cuvaju se samo reci duze od 2 karaktera u deep url
    word_list1 = [elem for elem in word_list if len(elem) > 2] 
    words_string = ' '.join(word_list1)
    return str(words_string)

In [17]:
ds_comb['deep url'] = ds_comb['deep url'].apply(lambda x : split_deep_url(x))
ds_comb.head()

,url,label,deep url,len deep url
0,http://aolshop.memberoffers.co.uk/csp/smp/aol/...,1,csp smp aol itementry3 csp E179 cat header Kit...,150
1,http://torcache.net/torrent/304E789D6E0E877BFD...,0,torrent 304E789D6E0E877BFD6070B73082DE6B1FF462...,138
2,http://amber.ch.ic.ac.uk/archive/all/21291.html,1,archive all 21291 html,23
3,http://www.gxyzx.com/js/?http://us.battle.net/...,1,http battle net login ref http 2Feqrwpzwus bat...,106
4,http://online.citicard.com.rpzmvfcek2.agencja-...,1,doctorsincanton com css online php,38


In [0]:
#primenjujemo CountVectorizer da bismo imali ulaz za MultinomialDB
x = ds_comb['deep url']
y = ds_comb['label']

In [19]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size=0.33, random_state=7, stratify=y)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(52869,)
(26040,)
(52869,)
(26040,)


In [20]:
from sklearn import feature_extraction
vectorized = feature_extraction.text.CountVectorizer()
x_train_t = vectorized.fit_transform(x_train)
vectorized.vocabulary_

{'pin': 51081,
 'create': 31159,
 'button': 28166,
 'url': 61914,
 'http': 39904,
 'www': 64304,
 'greeteasy': 38475,
 'com': 30318,
 'warm': 63477,
 'wishes': 64038,
 'for': 36738,
 'new': 47620,
 'year': 64608,
 'media': 45700,
 'zumvu': 65188,
 '2014': 6961,
 'blog': 27160,
 'photos': 50956,
 '20140121': 6965,
 '1390316369': 4195,
 '161': 4883,
 '1390316327': 4194,
 'png': 51367,
 'ip14m3xd2sehsdi0d3mx2imz': 41386,
 'detail': 32526,
 '416413': 14788,
 'society': 57503,
 'events': 35267,
 'mbb': 45599,
 'info': 40942,
 'm2u': 44883,
 'public': 52748,
 'personaldetail04': 50761,
 'channelid': 29415,
 'crd': 31143,
 'cards': 28823,
 'cnttypeid': 30138,
 'cntkey': 30137,
 'crd01': 31144,
 'programid': 52447,
 'ccfeatures': 29107,
 'chcatid': 29508,
 'personal': 50754,
 'recipe': 53662,
 'marinated': 45309,
 'pork': 51658,
 'tenderloin': 59898,
 'aspx': 24275,
 'soid': 57550,
 'recs': 53718,
 'index': 40841,
 'php': 50961,
 'manufacturers': 45235,
 'basicgrey': 25774,
 'list': 44306,
 's

In [21]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train_t, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
x_test_t = vectorized.transform(x_test)
x_test_t

<26040x65253 sparse matrix of type '<class 'numpy.int64'>'
	with 183037 stored elements in Compressed Sparse Row format>

In [0]:
y_pred = nb.predict(x_test_t)

In [24]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred)

0.9608294930875576

In [25]:
metrics.confusion_matrix(y_test, y_pred)

array([[11120,   555],
       [  465, 13900]])

In [26]:
#dobijamo predvidjene verovatnoce za pripadnost klasi 0 i 1, respektivno
#klasa 0 znaci label = 0 
y_pred_proba = nb.predict_proba(x_test_t)
y_pred_proba

array([[8.52039767e-05, 9.99914796e-01],
       [5.37201225e-03, 9.94627988e-01],
       [9.84468315e-04, 9.99015532e-01],
       ...,
       [2.47673629e-09, 9.99999998e-01],
       [6.85275223e-02, 9.31472478e-01],
       [2.48997358e-03, 9.97510026e-01]])

In [27]:
#tacnost predvidjanja za klasu 1
metrics.roc_auc_score(y_test, y_pred_proba[:, 1])

0.9897542310412757

In [0]:
y_pred_proba_0 = y_pred_proba[:, 0]
y_pred_proba_1 = y_pred_proba[:, 1]

In [29]:
#pravimo dataframe od x_test jer hocemo dobijene verovatnoce da koristimo kao feature za SVM i RF
df_test = x_test.to_frame()
df_test.head()

,deep url
15450,configurazione supporto apple store contatta A...
66594,login login html
43205,modules mod feed dropbox
6638,index php option com content view article Itemid
36827,robots txt


In [30]:
df_test['label_proba_0'] = y_pred_proba_0
df_test['label_proba_1'] = y_pred_proba_1
df_test.head()

,deep url,label_proba_0,label_proba_1
15450,configurazione supporto apple store contatta A...,8.520398e-05,0.999915
66594,login login html,5.372012e-03,0.994628
43205,modules mod feed dropbox,9.844683e-04,0.999016
6638,index php option com content view article Itemid,1.407942e-12,1.000000
36827,robots txt,8.082654e-03,0.991917


In [31]:
#radicemo join sa originalnim datasetom pa nam ne treba deep url kolona
df_test = df_test.drop('deep url', axis = 1)
df_test.head()

,label_proba_0,label_proba_1
15450,8.520398e-05,0.999915
66594,5.372012e-03,0.994628
43205,9.844683e-04,0.999016
6638,1.407942e-12,1.000000
36827,8.082654e-03,0.991917


In [32]:
#join sa originalnim datasetom - radi join po indeksu, a dodeljuje nan vrednosti u redovima koji nisu u x_test
final_ds = ds_comb.join(df_test)
final_ds.head()

,url,label,deep url,len deep url,label_proba_0,label_proba_1
0,http://aolshop.memberoffers.co.uk/csp/smp/aol/...,1,csp smp aol itementry3 csp E179 cat header Kit...,150,1.991278e-33,1.000000e+00
1,http://torcache.net/torrent/304E789D6E0E877BFD...,0,torrent 304E789D6E0E877BFD6070B73082DE6B1FF462...,138,1.000000e+00,4.366361e-16
2,http://amber.ch.ic.ac.uk/archive/all/21291.html,1,archive all 21291 html,23,NaN,NaN
3,http://www.gxyzx.com/js/?http://us.battle.net/...,1,http battle net login ref http 2Feqrwpzwus bat...,106,NaN,NaN
4,http://online.citicard.com.rpzmvfcek2.agencja-...,1,doctorsincanton com css online php,38,2.475952e-05,9.999752e-01


In [33]:
#izbacujemo redove sa nan vrednostima i resetujemo index da bi sad radili sa novim datasetom predikciju za SVM i RF
final_ds = final_ds.dropna()
final_ds = final_ds.reset_index(drop=True)
final_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26040 entries, 0 to 26039
Data columns (total 6 columns):
url              26040 non-null object
label            26040 non-null int64
deep url         26040 non-null object
len deep url     26040 non-null int64
label_proba_0    26040 non-null float64
label_proba_1    26040 non-null float64
dtypes: float64(2), int64(2), object(2)
memory usage: 1.2+ MB


In [0]:
#dodajemo kolone za duzinu domain-a i suffix

def get_len_of_domain_url(url):
  ext = tldextract.extract(url)
  return len(ext.domain)
    
def get_suffix_url(url):
  ext = tldextract.extract(url)
  return str(ext.suffix)

final_ds['len of domain'] = final_ds['url'].apply(lambda x : get_len_of_domain_url(x))
final_ds['suffix'] = final_ds['url'].apply(lambda x : get_suffix_url(x))

In [0]:
#suffix kolona (top level domain-tld u radu) ima previse razlicitih vrednosti i radi se cutoff za frekvenciju < 50  
minFreq = 50 
suffix_values = final_ds['suffix'].value_counts() # Specific column 
to_remove = suffix_values[suffix_values <= minFreq].index
final_ds['suffix'].replace(to_remove, np.nan, inplace=True)

In [36]:
#sada imamo nedostajuce vrednosti(nan) u koloni suffix, pa izbacujemo takve redove i resetujrmo index
final_ds = final_ds.dropna()
final_ds = final_ds.reset_index(drop=True)
final_ds['suffix'].value_counts()

com       11062
co.uk      3256
net        2124
info       1363
ac.uk       788
org         540
de          525
ru          465
vn          444
com.br      415
pl          397
cc          310
jp          259
nl          246
it          207
ro          197
net.ua      158
com.vn      151
org.uk      140
ua          137
cn          124
to          123
gov.uk      121
co.id       120
com.au      120
in          111
asia        107
gr           93
tv           88
net.vn       86
fr           84
ir           81
eu           79
com.cn       76
com.my       69
gov.br       64
             60
es           58
cl           57
Name: suffix, dtype: int64

In [37]:
#izbacujemo nepotrebne kolone (url, deep url) i enkodiramo kategoricku promenljivu suffix, kao u radu
final_ds = final_ds.drop(['url', 'deep url'], axis = 1)
final_ds = pd.get_dummies(final_ds, prefix = ['suffix'], columns = ['suffix'])
final_ds.head()

,label,len deep url,label_proba_0,label_proba_1,len of domain,suffix_,suffix_ac.uk,suffix_asia,suffix_cc,suffix_cl,suffix_cn,suffix_co.id,suffix_co.uk,suffix_com,suffix_com.au,suffix_com.br,suffix_com.cn,suffix_com.my,suffix_com.vn,suffix_de,suffix_es,suffix_eu,suffix_fr,suffix_gov.br,suffix_gov.uk,suffix_gr,suffix_in,suffix_info,suffix_ir,suffix_it,suffix_jp,suffix_net,suffix_net.ua,suffix_net.vn,suffix_nl,suffix_org,suffix_org.uk,suffix_pl,suffix_ro,suffix_ru,suffix_to,suffix_tv,suffix_ua,suffix_vn
0,1,150,1.991278e-33,1.000000e+00,12,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,138,1.000000e+00,4.366361e-16,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,1,38,2.475952e-05,9.999752e-01,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,1,53,2.815308e-02,9.718469e-01,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,22,1.373080e-02,9.862692e-01,14,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
#prebacujemo kolone u numericke
final_ds['len of domain'] = pd.to_numeric(final_ds['len of domain'])
final_ds['len deep url'] = pd.to_numeric(final_ds['len deep url'])
final_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 44 columns):
label            24905 non-null int64
len deep url     24905 non-null int64
label_proba_0    24905 non-null float64
label_proba_1    24905 non-null float64
len of domain    24905 non-null int64
suffix_          24905 non-null uint8
suffix_ac.uk     24905 non-null uint8
suffix_asia      24905 non-null uint8
suffix_cc        24905 non-null uint8
suffix_cl        24905 non-null uint8
suffix_cn        24905 non-null uint8
suffix_co.id     24905 non-null uint8
suffix_co.uk     24905 non-null uint8
suffix_com       24905 non-null uint8
suffix_com.au    24905 non-null uint8
suffix_com.br    24905 non-null uint8
suffix_com.cn    24905 non-null uint8
suffix_com.my    24905 non-null uint8
suffix_com.vn    24905 non-null uint8
suffix_de        24905 non-null uint8
suffix_es        24905 non-null uint8
suffix_eu        24905 non-null uint8
suffix_fr        24905 non-null uint8
suffix_gov.br

In [0]:
#uzimamo X i y za RandomForest
X_rf = final_ds.drop(['label'],axis=1) 
y_rf = final_ds['label']

In [40]:
#train test split
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.3, random_state=42)

# koristimo gini loss kao u radu
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=30, criterion = 'gini')
rf.fit(X_train_rf, y_train_rf)

#trening set skor
rf.score(X_train_rf, y_train_rf)

0.9997705501061206

In [41]:
y_pred_rf = rf.predict(X_test_rf)
#test set skor
accuracy_score(y_test_rf,rf.predict(X_test_rf))

0.9921038543897216

In [42]:
classification_report(y_test_rf,rf.predict(X_test_rf))

'              precision    recall  f1-score   support\n\n           0       0.99      0.99      0.99      3409\n           1       0.99      0.99      0.99      4063\n\n    accuracy                           0.99      7472\n   macro avg       0.99      0.99      0.99      7472\nweighted avg       0.99      0.99      0.99      7472\n'

In [43]:
confusion_matrix(y_test_rf,rf.predict(X_test_rf))

array([[3382,   27],
       [  32, 4031]])

In [44]:
#najbitniji atributi opadajuce
feature_importances = zip(list(X_rf), rf.feature_importances_)
feature_importances_sort = sorted(feature_importances, key=lambda x: x[1], reverse=True)

for feature in feature_importances_sort:
    print(feature)

('label_proba_0', 0.4222550932514802)
('label_proba_1', 0.3306371376897271)
('len deep url', 0.1377263589696134)
('len of domain', 0.03289329416455677)
('suffix_co.uk', 0.02123272662346501)
('suffix_info', 0.012657560209120688)
('suffix_com', 0.008009207455381369)
('suffix_net', 0.0068876253162152914)
('suffix_jp', 0.006205205492036044)
('suffix_ac.uk', 0.004735325924596362)
('suffix_tv', 0.0028700789306347362)
('suffix_vn', 0.0027928001247870117)
('suffix_ua', 0.00209053687232648)
('suffix_net.ua', 0.001031911105136378)
('suffix_nl', 0.0009796650193679365)
('suffix_cc', 0.0008868411195214441)
('suffix_com.au', 0.0006701907177895299)
('suffix_ir', 0.0006603408793445356)
('suffix_org.uk', 0.0006467735239897538)
('suffix_org', 0.0005262313566440535)
('suffix_pl', 0.0004321221458500497)
('suffix_ru', 0.00043150034081490145)
('suffix_de', 0.0004100138911869128)
('suffix_cn', 0.0002677575983821512)
('suffix_to', 0.00024296341859491016)
('suffix_com.br', 0.00022966038162217385)
('suffix_ro',

In [0]:
#ne moramo opet raditi train test split, a za svm nam treba standardizacija
X_train_svm = X_train_rf
y_train_svm = y_train_rf
X_test_svm = X_test_rf
y_test_svm = y_test_rf

In [46]:
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()
scaler.fit(X_train_svm)
scaler.transform(X_train_svm)
scaler.transform(X_test_svm)

array([[ 0.14945578,  1.08494047, -1.08494047, ..., -0.05727466,
        -0.07480169, -0.13232561],
       [-0.18659285, -1.06311355,  1.06311355, ..., -0.05727466,
        -0.07480169, -0.13232561],
       [-0.74236558,  0.3234284 , -0.3234284 , ..., -0.05727466,
        -0.07480169, -0.13232561],
       ...,
       [-0.10904317, -0.10006669,  0.10006669, ..., -0.05727466,
        -0.07480169, -0.13232561],
       [ 0.18823062,  1.08490375, -1.08490375, ..., -0.05727466,
        -0.07480169, -0.13232561],
       [-0.08319327, -1.06270623,  1.06270623, ..., -0.05727466,
        -0.07480169, -0.13232561]])

In [47]:
#linear SVM
linear_svm = svm.LinearSVC(loss='hinge', C=1.0)
linear_svm.fit(X_train_svm, y_train_svm)
linear_acc = metrics.accuracy_score(y_test_svm, linear_svm.predict(X_test_svm))
#accuracy score
linear_acc

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9670770877944326

In [48]:
classification_report(y_test_svm,linear_svm.predict(X_test_svm))

'              precision    recall  f1-score   support\n\n           0       0.95      0.98      0.96      3409\n           1       0.98      0.96      0.97      4063\n\n    accuracy                           0.97      7472\n   macro avg       0.97      0.97      0.97      7472\nweighted avg       0.97      0.97      0.97      7472\n'

In [49]:
confusion_matrix(y_test_svm,linear_svm.predict(X_test_svm))

array([[3343,   66],
       [ 180, 3883]])

In [50]:
#SVM sa rbf kernelom
rbf_svm = svm.SVC(kernel='rbf', random_state=0, gamma=0.01, C=1)
rbf_svm.fit(X_train_svm, y_train_svm)
rbf_acc = metrics.accuracy_score(y_test_svm, rbf_svm.predict(X_test_svm))
#accuracy score
rbf_acc

0.9785867237687366

In [51]:
classification_report(y_test_svm,rbf_svm.predict(X_test_svm))

'              precision    recall  f1-score   support\n\n           0       0.98      0.97      0.98      3409\n           1       0.98      0.98      0.98      4063\n\n    accuracy                           0.98      7472\n   macro avg       0.98      0.98      0.98      7472\nweighted avg       0.98      0.98      0.98      7472\n'

In [52]:
confusion_matrix(y_test_svm,rbf_svm.predict(X_test_svm))

array([[3315,   94],
       [  66, 3997]])

In [0]:
#prikazane su samo SVM i RF sa parametrima koji daju najvecu tacnost, isprobane su razlicite vrednosti